In [1]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import torch
import evaluate

# Llama 3 - 8B Instruct Model

In [2]:
model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, torch_dtype=torch.bfloat16, device_map="auto")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
messages = [
    {"role": "system", "content": f"""You are a AI writer who creates stories in various genres. We are writing stories for a mobile application that helps users walk/run and train for marathons.  
The genre and description will come by user input called Prompt.
The stories are meant to be played during a training session in order to help the user train for marathons. 
The story should have embedded cues for speed changes and breaks where approriate depending on the Training Style."""},
    {"role": "user", "content": f"""Can you help me write a story with the following details:
Prompt: A zombie horror story set in New York City.
Distance: 0.5 miles
Average Walking Speed: 2 mph
Average Running Speed: 5 mph
Training Style: Short-distance sprint"""},
]

input_ids = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt=True,
    return_tensors="pt"
).to(model.device)

terminators = [
    tokenizer.eos_token_id,
    tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [4]:
outputs = model.generate(
    input_ids,
    max_new_tokens=256,
    eos_token_id=terminators,
    do_sample=True,
    temperature=0.6,
    top_p=0.9,
)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.
c:\Users\pmahi\OneDrive\Desktop\MP\Drexel\Courses\CS614-900 - Applications in Machine Learning\Projects\Natural_Language_Assignment\.venv\Lib\site-packages\transformers\models\llama\modeling_llama.py:649: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at ..\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


In [5]:
response = outputs[0][input_ids.shape[-1]:]
print(tokenizer.decode(response, skip_special_tokens=True))

Here's a zombie horror story set in New York City, tailored to your requirements:

**Title:** "Escape from Times Square"

You've always loved the bright lights and bustling energy of Times Square, but today it's a different story. A zombie outbreak has swept through the city, and you're one of the few survivors. Your goal is to make it to the safety of Central Park, where a rumored evacuation point is waiting.

You're standing in the middle of Times Square, surrounded by towering skyscrapers and endless streams of panicked pedestrians. The streets are chaos, with zombies shuffling and stumbling through the crowds. You can hear the distant moans and screams, and the air is thick with the stench of death.

**Cue:** Start walking at 2 mph (Average Walking Speed)

As you begin your journey, you notice a group of zombies shambling towards you. They're slow, but there are a lot of them. You need to keep moving to avoid getting overwhelmed.

You weave through the crowds, dodging and weaving a

In [3]:
# perplexity = evaluate.load("perplexity", module_type="metric")
# input_texts = ["lorem ipsum", "Happy Birthday!", "Bienvenue"]
# results = perplexity.compute(model_id=model_id,
#                              add_start_token=False,
#                              predictions=input_texts, 
#                              device="cuda")
# print(list(results.keys()))

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# print(round(results["mean_perplexity"], 2))